#  Configuração do Catálogo e Estrutura Inicial

> **Nota:**  
> Devido à instrução de que **apenas dois notebooks** poderiam ser enviados para esta atividade correspondentes às camadas **Bronze** e **Silver** do modelo de arquitetura **Medalhão**, a etapa de **setup inicial** (criação de catálogo, schemas e volume) foi incluída neste notebook **`01_land_to_bronze`**.  
>
> Em um ambiente de produção, ou caso houvesse permissão para o envio de um terceiro arquivo, essa etapa ficaria em um notebook separado de configuração (por exemplo, `00_setup_environment.ipynb`), executado uma única vez antes do fluxo de ingestão.  
>
> No entanto, para garantir a **reprodutibilidade** e o **funcionamento completo** do pipeline apenas com os dois arquivos exigidos, optou-se por incluir aqui os comandos:
> - `CREATE CATALOG IF NOT EXISTS medalhao;`
> - `USE CATALOG medalhao;`
> - `CREATE SCHEMA IF NOT EXISTS bronze;`
> - `CREATE SCHEMA IF NOT EXISTS silver;`
> - `USE SCHEMA default; CREATE VOLUME IF NOT EXISTS landing;`

## Setup Inicial

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS medalhao;

In [0]:
%sql
USE CATALOG medalhao;
    
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;

In [0]:
%sql
USE SCHEMA default;
CREATE VOLUME IF NOT EXISTS landing;

## CONFIGURAÇÕES INICIAIS E INGESTÃO

**Objetivo:**  
Esta célula importa as bibliotecas essenciais para a manipulação de dados, conexão com o Spark e integração com APIs externas.  

**Descrição dos principais imports:**
- `pyspark.sql` → Criação de sessões Spark e manipulação de DataFrames distribuídos.  
- `datetime` → Manipulação de datas e formatação de períodos.  
- `pyspark.sql.functions` → Funções nativas do Spark, como `current_timestamp`, `col`, `max` e agregações diversas.  
- `requests` → Realiza requisições HTTP para a API do Banco Central (cotação do dólar).  
- `pandas` → Suporte à conversão de dados entre estruturas locais (Pandas DataFrame) e distribuídas (Spark DataFrame).  
- `time` → Utilizado para controlar intervalos e tentativas em requisições com retries.  


In [0]:
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import current_timestamp, col, max
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import requests
import time
import pandas as pd

**Objetivo:**  
Definir as variáveis básicas da arquitetura Medalhão e apontar o ambiente Spark para o catálogo e schema corretos antes das operações de leitura e escrita.

**O que a célula faz:**
- `catalog_name`, `bronze_db_name`, `silver_db_name` → definem os nomes das camadas lógicas da arquitetura Medalhão.  
- `landing_volume` → representa o volume físico usado para armazenar os arquivos brutos (camada *Landing*).  
- `base_path` → constrói dinamicamente o caminho para o volume de origem dentro do Unity Catalog (`/Volumes/medalhao/default/landing`).  
- `spark.sql("USE CATALOG ...")` → muda o contexto do Spark para o catálogo principal `medalhao`.  
- `spark.sql("USE SCHEMA ...")` → seleciona o schema `bronze`, garantindo que todas as tabelas criadas a partir deste ponto sejam registradas nessa camada.  


In [0]:
catalog_name = "medalhao"
bronze_db_name = "bronze"
silver_db_name = "silver"

landing_volume = "landing"

base_path = f"/Volumes/{catalog_name}/default/{landing_volume}"

spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {bronze_db_name}")

**Objetivo:**  
Ler os arquivos CSV do volume *Landing*, adicionar a coluna de controle `ingestion_timestamp` e gravar os dados na camada **Bronze** da arquitetura Medalhão, criando as tabelas correspondentes no catálogo `medalhao`.

**Descrição detalhada:**
- **`file_path`** → monta o caminho completo até o arquivo CSV dentro do volume `/Volumes/medalhao/default/landing`.  
- **`full_table_name`** → define o nome completo da tabela no formato `<catálogo>.<schema>.<tabela>`.  
- **`spark.read.format("csv")`** → lê o arquivo CSV com cabeçalho e infere automaticamente o schema das colunas.  
- **`withColumn("ingestion_timestamp", current_timestamp())`** → adiciona uma coluna com o instante exato de carregamento do dado, útil para auditoria e controle de versões.  
- **`df.write.mode("overwrite").saveAsTable(full_table_name)`** → grava os dados como uma **tabela gerenciada** no Unity Catalog, substituindo o conteúdo anterior (modo *overwrite*).  
- O `try/except` garante que possíveis erros de leitura ou escrita sejam tratados e exibidos no log.

**Observações:**
- As tabelas são salvas no formato **Delta** por padrão (Databricks gerencia automaticamente `saveAsTable` como Delta).  

In [0]:
def ingest_csv(file_name: str, table_name: str):
    file_path = f"{base_path}/{file_name}"
    full_table_name = f"{catalog_name}.{bronze_db_name}.{table_name}"
    
    print(f"\nIniciando ingestão de {file_name} → {full_table_name}")
    
    try:
        df = (
            spark.read
            .format("csv")
            .option("header", "true")
            .option("inferSchema", "true")
            .load(file_path)
        )
        df = df.withColumn("ingestion_timestamp", current_timestamp())
        
        df.write.mode("overwrite").saveAsTable(full_table_name)
        
        print(f"Tabela criada: {full_table_name} ({df.count()} registros)")
    
    except Exception as e:
        print(f"Erro ao ingerir {file_name}: {e}")

ingest_csv("olist_customers_dataset.csv", "ft_consumidores")
ingest_csv("olist_geolocation_dataset.csv", "ft_geolocalizacao")
ingest_csv("olist_order_items_dataset.csv", "ft_itens_pedidos")
ingest_csv("olist_order_payments_dataset.csv", "ft_pagamentos_pedidos")
ingest_csv("olist_order_reviews_dataset.csv", "ft_avaliacoes_pedidos")
ingest_csv("olist_orders_dataset.csv", "ft_pedidos")
ingest_csv("olist_products_dataset.csv", "ft_produtos")
ingest_csv("olist_sellers_dataset.csv", "ft_vendedores")
ingest_csv("product_category_name_translation.csv", "dm_categoria_produtos_traducao")

print("\nIngestão da camada Bronze concluída")

**Objetivo:**  
Permitir uma verificação rápida das tabelas ingeridas na camada Bronze, exibindo uma amostra dos dados carregados a partir dos arquivos CSV.

**Descrição da função:**
- **`preview_tables()`** → percorre a lista de tabelas definidas em `bronze_tables` e exibe as primeiras linhas de cada uma.  
- **Parâmetro `limit_rows`** → define quantas linhas serão exibidas (por padrão, 5).  
- **`spark.table(full_table_name)`** → lê cada tabela diretamente do catálogo `medalhao.bronze`.  
- **`display(df)`** → exibe os registros no formato visual interativo do Databricks, facilitando a inspeção manual.  
- O bloco `try/except` garante que, se alguma tabela não existir ou ocorrer erro de leitura, o processo continue normalmente para as demais.

In [0]:
def preview_tables(tables: list, limit_rows: int = 5):
    """
    Lê cada tabela da camada Bronze e exibe as primeiras linhas (limit configurável).
    """
    for table in tables:
        full_table_name = f"{catalog_name}.{bronze_db_name}.{table}"
        print(f"\n📘 Exibindo primeiras {limit_rows} linhas da tabela: {full_table_name}")
        try:
            df = spark.table(full_table_name).limit(limit_rows)
            display(df)
        except Exception as e:
            print(f"Erro ao exibir {full_table_name}: {e}")

bronze_tables = [
    "ft_consumidores",
    "ft_geolocalizacao",
    "ft_itens_pedidos",
    "ft_pagamentos_pedidos",
    "ft_avaliacoes_pedidos",
    "ft_pedidos",
    "ft_produtos",
    "ft_vendedores",
    "dm_categoria_produtos_traducao"
]

preview_tables(bronze_tables)


**Objetivo:**  
Preparar as variáveis e formatos de data que serão utilizados na etapa de ingestão da cotação do dólar, obtida via API do Banco Central.

**Descrição das variáveis:**
- **`table_full_name`** → define o nome completo da tabela que armazenará as cotações, seguindo o padrão do catálogo e schema do projeto (`medalhao.bronze.dm_cotacao_dolar`).  
  Essa tabela será responsável por armazenar as informações de cotação diária do dólar em formato Delta.  
- **`API_OUTPUT_FORMAT`** → especifica o formato exigido pela API do Banco Central (`%m-%d-%Y`), ou seja, *mês-dia-ano*.  
- **`POSSIBLE_INPUT_FORMATS`** → lista de formatos alternativos aceitos como entrada (ex.: `YYYY-MM-DD`, `DD/MM/YYYY`, `MM-DD-YYYY`). Esses formatos serão utilizados para converter automaticamente as datas antes da chamada à API.

In [0]:
table_full_name = f"{catalog_name}.{bronze_db_name}.dm_cotacao_dolar"

API_OUTPUT_FORMAT = "%m-%d-%Y" 
POSSIBLE_INPUT_FORMATS = ["%Y-%m-%d", "%d/%m/%Y", "%m-%d-%Y", "%Y/%m/%d"]

**Objetivo:**  
Garantir que as datas de início e fim utilizadas na consulta da API do Banco Central (PTAX) estejam no formato correto (`MM-DD-YYYY`), conforme exigido pelo endpoint oficial.

**Descrição da função:**
- **`normalize_widget_date(date_str)`** → tenta converter o valor de data recebido de um *widget* ou variável manual para o formato exigido pela API.  
  - Se a data estiver vazia, lança um `ValueError`.  
  - Se o formato não corresponder a nenhum dos esperados em `POSSIBLE_INPUT_FORMATS`, também gera um erro informativo.  
  - Em caso de sucesso, retorna a data convertida no formato `MM-DD-YYYY`.

**Fluxo de execução da célula:**
1. Define a data inicial (`raw_start_date = "2010-01-01"`) e a data final como a **data atual do sistema** (`datetime.now()`).
2. Exibe no console as datas utilizadas para a extração completa (*modo histórico total*).
3. Converte ambas as datas usando a função `normalize_widget_date()`.

In [0]:
def normalize_widget_date(date_str: str) -> str:
    """Tenta converter a string de data de entrada para o formato da API (MM-DD-YYYY)."""
    if not date_str:
        raise ValueError(f"Erro: O valor do widget está vazio ('{date_str}').")
        
    for fmt in POSSIBLE_INPUT_FORMATS:
        try:
            dt_obj = datetime.strptime(date_str, fmt)
            return dt_obj.strftime(API_OUTPUT_FORMAT)
        except ValueError:
            continue
            
    raise ValueError(
        f"Erro: O valor do widget '{date_str}' não corresponde a nenhum dos formatos de data esperados: "
        f"{', '.join(POSSIBLE_INPUT_FORMATS)}"
    )

raw_start_date = "2010-01-01" 
print(f"Modo Histórico Total: Iniciando extração em: {raw_start_date}")

raw_end_date = datetime.now().strftime("%Y-%m-%d")
print(f"Data Final: Usando data atual ({raw_end_date})")

try:
    start_date_formatted = normalize_widget_date(raw_start_date)
    end_date_formatted = normalize_widget_date(raw_end_date)
    
    print(f"\nPeríodo de Extração (API BCB):")
    print(f"  Data de Início: {start_date_formatted}")
    print(f"  Data Final:    {end_date_formatted}")
    
except ValueError as e:
    raise ValueError(f"Falha na conversão de data: {e}")

**Objetivo:**  
Conectar-se à API oficial do Banco Central do Brasil (PTAX) para obter as cotações de compra do dólar dentro do intervalo de datas previamente configurado.

**Descrição detalhada:**
- **`ENDPOINT`** → constrói dinamicamente a URL da requisição, inserindo as datas de início e fim formatadas conforme o padrão exigido pela API (`MM-DD-YYYY`).  
- **Requisição HTTP** → utiliza `requests.get()` com tempo limite de 30 segundos para garantir resposta dentro de um intervalo seguro.  
  Caso ocorra erro de conexão, `raise_for_status()` força a interrupção e o tratamento da exceção.  
- **`payload`** → armazena o conteúdo JSON retornado pela API. Em seguida, o código extrai os registros em `payload["value"]`.  
- **`df_pd`** → converte o conteúdo obtido em um `DataFrame` do pandas.  
- Se **não houver dados**, um `Spark DataFrame` vazio é criado com as colunas `dataHoraCotacao` e `cotacaoCompra`.  
- Se **houver dados**, a coluna `cotacaoCompra` é renomeada, e adiciona-se a coluna `ingestion_timestamp` para registrar o momento da ingestão.  

In [0]:
ENDPOINT = (
    "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
    f"CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?"
    f"@dataInicial='{start_date_formatted}'&@dataFinalCotacao='{end_date_formatted}'"
    f"&$select=dataHoraCotacao,cotacaoCompra&$format=json"
)

try:
    resp = requests.get(ENDPOINT, timeout=30)
    resp.raise_for_status()
except requests.exceptions.RequestException as e:
    raise RuntimeError(f"Falha ao acessar a API do BCB: {e}")

payload = resp.json()
records = payload.get("value", [])

if not records:
    print("API retornou 0 registros para o período. Nenhuma linha será inserida.")

df_pd = pd.DataFrame(records)

if df_pd.shape[0] == 0:
    spark_df = spark.createDataFrame([], schema='dataHoraCotacao:string, cotacaoCompra:string')
    print("Nenhum dado novo para carregar.")
else:
    if "cotacaoCompra" in df_pd.columns:
        df_pd = df_pd.rename(columns={"cotacaoCompra": "purchase_rate"})
    
    if "dataHoraCotacao" not in df_pd.columns:
         df_pd["dataHoraCotacao"] = None 

    spark_df = spark.createDataFrame(df_pd)
    spark_df = spark_df.withColumn("ingestion_timestamp", current_timestamp())
    
    print(f"Dados extraídos. Total de linhas prontas para carga: {spark_df.count()}")
    spark_df.display()

**Objetivo:**  
Armazenar os dados extraídos da API do Banco Central na tabela `dm_cotacao_dolar` da camada Bronze, utilizando o formato Delta para garantir rastreabilidade e versionamento dos dados.

**Descrição detalhada:**
- **Verificação inicial:** caso o `DataFrame` esteja vazio (`spark_df.count() == 0`), o processo é encerrado exibindo uma mensagem informativa de que não há novas cotações para carregar.  
- **Gravação em Delta:** se houver dados, o `DataFrame` é salvo com:
  - **`format("delta")`** → define o formato Delta Lake, padrão para camadas do Lakehouse;  
  - **`mode("overwrite")`** → sobrescreve completamente os dados anteriores, mantendo o schema atualizado;  
  - **`option("overwriteSchema", "true")`** → assegura que alterações no schema sejam refletidas na tabela existente;  
  - **`saveAsTable(table_full_name)`** → grava os dados como tabela gerenciada no catálogo `medalhao.bronze`.  

In [0]:
if spark_df.count() == 0:
    print("Processo finalizado: Nenhuma nova cotação para carregar na Bronze.")
else:
    spark_df.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(table_full_name)

    print(f"Dados da cotação salvos em {table_full_name} (novas linhas: {spark_df.count()})")

    spark.table(table_full_name).orderBy(col("dataHoraCotacao").desc()).limit(10).display()